# Trabajo Práctico 2: Enunciado


El segundo TP es una competencia de Machine Learning en donde cada grupo debe intentar determinar, para cada tweet brindado, si el mismo esta basado en un hecho real o no.

La competencia se desarrolla en la plataforma de Kaggle  https://www.kaggle.com/c/nlp-getting-started.  

El dataset consta de una serie de tweets, para los cuales se informa:

- id - identificador unico para cada  tweet
- text - el texto del tweet
- location - ubicación desde donde fue enviado (podría no estar)
- keyword - un keyword para el tweet  (podría faltar)
- target - en train.csv, indica si se trata de un desastre real  (1) o no (0) 

Los submits con el resultado deben tener el formato:
- Id: Un id numérico para identificar el tweet
- target: 1 / 0 según se crea que el tweet se trata sobre un desastre real, o no.

Los grupos deberán probar distintos algoritmos de Machine Learning para intentar predecir si el tweet está basado en hechos reales o no. A medida que los grupos realicen pruebas deben realizar el correspondiente submit en Kaggle para evaluar el resultado de los mismos.

Al finalizar la competencia el grupo que mejor resultado tenga obtendrá 10 puntos para cada uno de sus integrantes que podrán ser usados en el examen por promoción o segundo recuperatorio.

Requisitos para la entrega del TP2:

- El TP debe programarse en Python o R.
- Debe entregarse un pdf con el informe de algoritmos probados, algoritmo final utilizado, transformaciones realizadas a los datos, feature engineering, etc. 
- El informe debe incluir también un link a github con el informe presentado en pdf, y todo el código.
- El grupo debe presentar el TP en una computadora en la fecha indicada por la cátedra, el TP debe correr en un lapso de tiempo razonable (inferior a 1 hora) y generar un submission válido que iguale el mejor resultado obtenido por el grupo en Kaggle. (mas detalles a definir)

El TP2 se va a evaluar en función del siguiente criterio:

- Cantidad de trabajo (esfuerzo) del grupo: ¿Probaron muchos algoritmos? ¿Hicieron un buen trabajo de pre-procesamiento de los datos y feature engineering?
- Resultado obtenido en Kaggle (obviamente cuanto mejor resultado mejor nota)
- Presentación final del informe, calidad de la redacción, uso de información obtenida en el TP1, conclusiones presentadas.
- Performance de la solución final.

In [ ]:
# Instalación de librerías a usar
# para escapear html
!pip install bs4
# para normalización del texto
!pip install nltk
!pip install stopwords

In [ ]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import string

from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
from html import unescape
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
#Para generar una paleta de colores equivalente a cubehelix(sns) en matplotlib
from matplotlib.colors import ListedColormap

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

#Nueva paleta de colores para matplotlib
color_hex_arr = ["#2B6F39", "#C2D8F2","#182D48","#A1784A","#D38FC5"]
cubehelix_map = ListedColormap(sns.color_palette(color_hex_arr).as_hex())

warnings.filterwarnings('ignore')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

In [ ]:
train = pd.read_csv('../data/train.csv', encoding='utf-8')
train.info()

In [ ]:
test = pd.read_csv('../data/test.csv', encoding='utf-8')
test.info()

### Quitamos la columna Location
Del análisis del tp1 vimos que la columna location tenía muchos valores nulos y muchísimos valores únicos, muchas veces con textos sin sentido

In [ ]:
test.drop(['location'], axis=1)
train.drop(['location'], axis=1)

In [ ]:
# creamos el daset tweets para después poder aplicarle todas las tranformaciones juntas, después volvemos a dividir
tweets = pd.concat([train, test])

## Agregado de nuevas columnas y limpieza del dataset

In [ ]:
# primera normalización del texto, pasamos a lowercase 
tweets['normalized_text'] = tweets.text.str.lower()

### Normalización del texto y nuevas columnas a partir del texto

#### Eliminamos los links del texto normalizado y guardamos los links en una columna a parte por si sirven a futuro

In [ ]:
URLPATTERN = r'(https?://\S+)' 

tweets['urls'] = tweets.normalized_text.apply(lambda x: re.findall(URLPATTERN, x))
tweets['normalized_text'] = tweets.normalized_text.apply(lambda x: re.sub(URLPATTERN,"", x))
# cuento la cantidad de links en los tweets
tweets['url_count'] = tweets.urls.str.len()

#### Eliminamos los hashtags del texto normalizado y lo dejamos en otra columna junto con el count

In [ ]:
# cuento la cantidad de hashtags en los tweets
# nueva columna con el total de hashtags, y los hashtags
tweets['hashtags'] = tweets.normalized_text.apply(lambda x: re.findall(r"#(\w+)", x))
tweets['hashtags_count'] = tweets.hashtags.str.len()
tweets['normalized_text'] = tweets.normalized_text.apply(lambda x: re.sub(r"#(\w+)","", x))

#### Eliminamos los tags del texto y lo dejamos en otra columna junto con el count

In [ ]:
# cuento la cantidad de ags en los tweets
# nueva columna con el total de tags, y los tags
tweets['tags'] = tweets.text.str.lower().apply(lambda x: re.findall(r"@(\w+)", x))
tweets['normalized_text'] = tweets.normalized_text.apply(lambda x: re.sub(r"@(\w+)","", x))
tweets['tags_count'] = tweets.tags.str.len()

#### Contamos cantidad de oraciones

In [ ]:
tweets['sentences_count'] = tweets.normalized_text.apply(lambda x : len(sent_tokenize(x))) 

#### Eliminamos signos de puntuación y html

In [ ]:
translator = str.maketrans('', '', string.punctuation) 
# !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~

def remove_punctuation(text):        
    return text.translate(translator)

In [ ]:
def remove_htmlsymbols(text):
    soup = BeautifulSoup(unescape(text))
    return soup.text

In [ ]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

def remove_emojis_non_ascii(text):    
    #replace consecutive non-ASCII characters with a space
    result = re.sub(r'[^\x00-\x7F]+',' ', text)
    #remove emojis from tweet
    result = emoji_pattern.sub(r'', result)    
    return result

In [ ]:
tweets['normalized_text'] = tweets.normalized_text.apply(remove_htmlsymbols)
tweets['normalized_text'] = tweets.normalized_text.apply(remove_punctuation)
tweets['normalized_text'] = tweets.normalized_text.apply(remove_emojis_non_ascii)

#### Eliminamos stop words y creamos nueva columna con array de palabras

In [ ]:
# convertimos el texto en listado de palabras y despues borramos las stop words
tweets['words'] = tweets.normalized_text.str.split()
stop_words = stopwords.words('english')
tweets['normalized_words'] = tweets['words'].apply(lambda x: [item for item in x if item not in stop_words])
# eliminar las stop words del texto normalizado
tweets['normalized_text'] = [' '.join(map(str, l)) for l in tweets['normalized_words']]
# vemos como queda el dataset
tweets.head()

#### Contador de palabras y frecuencia de aparición

In [ ]:
# contamos la cantidad de palabras de cada tweet
tweets['words_counter'] = tweets.normalized_words.apply(Counter)
# me quedo con las palabras que mas ocurrencias tienen en cada row
tweets['word_max_appearance'] = tweets.words_counter.apply( lambda x: max(x) if x else None)  
tweets['word_max_appearance_count'] =  tweets.words_counter.apply( lambda x: max(x.values()) if x else 0)

#### Analizamos si el texto tiene números y guardamos el dato en una nueva columna

In [ ]:
# esto chequea números como 1,2 y tambíen escrito como one, two
def existence_of_numeric_data(text):
    text=nltk.word_tokenize(text)
    pos = nltk.pos_tag(text)
    count = 0
    for i in range(len(pos)):
        word , pos_tag = pos[i]
        if pos_tag == 'CD':
            return 1
    return 0

In [ ]:
# con 1 si tiene , 0 sino
tweets['has_numbers'] = tweets.normalized_text.apply(existence_of_numeric_data)

### Normalización de la columna keyword

In [ ]:
# eliminamos "%20" que representa espacio
tweets['keywords'] = tweets.keyword.str.replace('%20',' ')

### Correccion tipo de datos en keyword para optimizacion de memoria.

In [ ]:
for col in ['keyword']:
    tweets[col] = tweets[col].astype('category')
#check de tipos de datos categoricos 
tweets.dtypes

### Agregado de columnas para Análisis de sentimientos

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
# Return a float for sentiment strength based on the input text. Positive values are positive valence, negative value are negative valence.
tweets['sentiment_score'] = tweets.normalized_text.apply(lambda x: sid.polarity_scores(x))
tweets['sentiment_score_compound'] = tweets.sentiment_score.apply(lambda x: x['compound'])
tweets['sentiment_score_pos'] = tweets.sentiment_score.apply(lambda x: x['pos'])
tweets['sentiment_score_neg'] = tweets.sentiment_score.apply(lambda x: x['neg'])

In [ ]:
# vemos cómo queda el dataset
tweets.head()

### Volvemos a separar los sets de datos en train y test

In [ ]:
test = pd.concat([tweets.set_index('id'),test.set_index('id')], axis=1, join='inner')


In [ ]:
test.info()

In [ ]:
train = pd.concat([tweets.set_index('id'),train.set_index('id')], axis=1, join='inner')


In [ ]:
train.info()